## HW3: Decision Tree, AdaBoost and Random Forest
In hw3, you need to implement decision tree, adaboost and random forest by using only numpy, then train your implemented model by the provided dataset and test the performance with testing data

Please note that only **NUMPY** can be used to implement your model, you will get no points by simply calling sklearn.tree.DecisionTreeClassifier

## Load data
The dataset is the Heart Disease Data Set from UCI Machine Learning Repository. It is a binary classifiation dataset, the label is stored in `target` column. **Please note that there exist categorical features which need to be [one-hot encoding](https://www.datacamp.com/community/tutorials/categorical-data) before fit into your model!**
See follow links for more information
https://archive.ics.uci.edu/ml/datasets/heart+Disease

In [1]:
import pandas as pd
import numpy as np
file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
df = pd.read_csv(file_url)

train_idx = np.load('train_idx.npy')
test_idx = np.load('test_idx.npy')

train_df = df.iloc[train_idx]
test_df = df.iloc[test_idx]

In [2]:
train_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
264,44,1,4,112,290,0,2,153,0,0.0,1,1,normal,1
140,62,0,4,138,294,1,0,106,0,1.9,2,3,normal,1
104,70,1,4,130,322,0,2,109,0,2.4,2,3,normal,0
290,49,1,3,120,188,0,0,139,0,2.0,2,3,reversible,1
262,41,0,2,105,198,0,0,168,0,0.0,1,1,normal,0


## Question 1
Gini Index or Entropy is often used for measuring the “best” splitting of the data. Please compute the Entropy and Gini Index of provided data. Please use the formula from [page 5 of hw3 slides](https://docs.google.com/presentation/d/1kIe_-YZdemRMmr_3xDy-l0OS2EcLgDH7Uan14tlU5KE/edit#slide=id.gd542a5ff75_0_15)

In [3]:
def gini(sequence):
    unique, counts = np.unique(sequence, return_counts=True)
    sum = 0
    for c in counts:
        # print(c/np.sum(counts))
        sum += (c/np.sum(counts))**2
    g = 1 - sum
    return g

def entropy(sequence):
    unique, counts = np.unique(sequence, return_counts=True)
    sum = 0
    for c in counts:
        # print(c/np.sum(counts))
        sum += (c/np.sum(counts))*np.log2(c/np.sum(counts))
    return sum*(-1)

In [4]:
# 1 = class 1,
# 2 = class 2
data = np.array([1,2,1,1,1,1,2,2,1,1,2])

In [5]:
print("Gini of data is ", gini(data))

Gini of data is  0.4628099173553719


In [6]:
print("Entropy of data is ", entropy(data))

Entropy of data is  0.9456603046006401


## Question 2
Implement the Decision Tree algorithm (CART, Classification and Regression Trees) and trained the model by the given arguments, and print the accuracy score on the test data. You should implement two arguments for the Decision Tree algorithm
1. **criterion**: The function to measure the quality of a split. Your model should support `gini` for the Gini impurity and `entropy` for the information gain. 
2. **max_depth**: The maximum depth of the tree. If `max_depth=None`, then nodes are expanded until all leaves are pure. `max_depth=1` equals to split data once


In [54]:
# unique_vals: set(pd.unique(train_df['cp']))
# class_counts: train_df['cp'].value_counts().to_dict()

cat_var = [col for col in df.columns.tolist() if len(df[col].unique()) <=5]
cont_var = [col for col in df.columns.tolist() if len(df[col].unique()) > 5]

print(train_df['target'].value_counts().to_dict())

{0: 147, 1: 56}


In [26]:
train_df.columns.tolist()
# train_df.values.tolist()

class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        col = train_df.columns.tolist()
        val = example[col.index(self.column)]
        if self.column in set(cont_var):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if self.column in set(cont_var):
            condition = ">="
        return "Is %s %s %s?" % (
            self.column, condition, str(self.value))

In [25]:
train_df.values[0]

array([44, 1, 4, 112, 290, 0, 2, 153, 0, 0.0, 1, 1, 'normal', 1],
      dtype=object)

In [53]:
training_data = train_df.values.tolist()
print(len(training_data))
for item in training_data:
    if item[12] == '1' or item[12] == '2':
        print(item)
        training_data.remove(item)
print(len(training_data))
q = Question('thal', 'normal')
example = training_data[0]
q.match(example)

203
[57, 0, 1, 130, 236, 0, 0, 174, 0, 0.0, 1, 1, '2', 0]
[59, 1, 0, 164, 176, 1, 0, 90, 0, 1.0, 1, 2, '1', 0]
201


True

In [29]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [ ]:
true_rows, false_rows = partition(training_data, Question('trestbps', 115))
print(true_rows)

In [31]:
print(false_rows)

[[44, 1, 4, 112, 290, 0, 2, 153, 0, 0.0, 1, 1, 'normal', 1], [41, 0, 2, 105, 198, 0, 0, 168, 0, 0.0, 1, 1, 'normal', 0], [57, 1, 4, 110, 201, 0, 0, 126, 1, 1.5, 2, 0, 'fixed', 0], [52, 1, 4, 108, 233, 1, 0, 147, 0, 0.1, 1, 3, 'reversible', 0], [43, 1, 4, 110, 211, 0, 0, 161, 0, 0.0, 1, 0, 'reversible', 0], [44, 1, 4, 110, 197, 0, 2, 177, 0, 0.0, 1, 1, 'normal', 0], [47, 1, 4, 110, 275, 0, 2, 118, 1, 1.0, 2, 1, 'normal', 0], [42, 0, 4, 102, 265, 0, 2, 122, 0, 0.6, 2, 0, 'normal', 0], [51, 1, 3, 94, 227, 0, 0, 154, 1, 0.0, 1, 1, 'reversible', 0], [58, 1, 3, 105, 240, 0, 2, 154, 1, 0.6, 2, 0, 'reversible', 0], [41, 1, 4, 110, 172, 0, 2, 158, 0, 0.0, 1, 0, 'reversible', 0], [45, 1, 1, 110, 264, 0, 0, 132, 0, 1.2, 2, 0, 'reversible', 0], [41, 1, 3, 112, 250, 0, 0, 179, 0, 0.0, 1, 0, 'normal', 0], [59, 1, 4, 110, 239, 0, 2, 142, 1, 1.2, 2, 1, 'reversible', 1], [41, 0, 3, 112, 268, 0, 2, 172, 1, 0.0, 1, 0, 'normal', 0], [63, 0, 4, 108, 269, 0, 0, 169, 1, 1.8, 2, 2, 'normal', 0], [51, 1, 3, 11

In [35]:
current_uncertainty = gini(training_data)
print(current_uncertainty)

0.8935023110168626


In [36]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [37]:
true_rows, false_rows = partition(training_data, Question('trestbps', 115))
info_gain(true_rows, false_rows, current_uncertainty)

0.0004064027027985695

In [89]:
def find_best_split(rows, measurment):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    if measurment == 'gini':
        current_uncertainty = gini(rows)
    elif measurment == 'entropy':
        current_uncertainty = entropy(rows)
    else:
        print('error')
        return

    for col in train_df.columns.tolist()[:-1]:  # for each feature
        total_col = train_df.columns.tolist()[:-1]
        values = set([row[total_col.index(col)] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [90]:
best_gain, best_question = find_best_split(training_data, 'gini')
print(best_question)

Is ca == 0?


In [91]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [92]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [93]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [97]:
def build_tree(rows, criterion, depth):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
    
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows, criterion)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0 or depth == 0:
        return Leaf(rows)
    
    depth -= 1
    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows, criterion, depth)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows, criterion, depth)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [98]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [104]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [138]:
testing_data = test_df.values.tolist()

def test_acc(testing_data, tree):
    total = len(testing_data)
    count = 0
    for row in testing_data:
        the_dict = classify(row, tree)
        # print(row[-1], classify(row, tree), list(the_dict.keys()))
        if len(list(the_dict.keys())) == 1:
            choose_class = list(the_dict.keys())[0]
        else:
            # print(the_dict[0], the_dict[1])
            if the_dict[0] > the_dict[1]:
                choose_class = 0
            else:
                choose_class = 1
        if choose_class == row[-1]:
            count += 1
    return count/total

In [10]:
'''
class DecisionTree():
    def __init__(self, criterion='gini', max_depth=None):
        self.max_depth = max_depth
        self.criterion = criterion
        self.tree = None
        return None
'''

### Question 2.1
Using `criterion=gini`, showing the accuracy score of test data by `max_depth=3` and `max_depth=10`, respectively.


In [11]:
# clf_depth3 = DecisionTree(criterion='gini', max_depth=3)
# clf_depth10 = DecisionTree(criterion='gini', max_depth=10)

In [107]:
clf_depth3 = build_tree(training_data, 'gini', 3)
clf_depth10 = build_tree(training_data, 'gini', 10)

In [139]:
print(test_acc(testing_data, clf_depth3))
print(test_acc(testing_data, clf_depth10))

0.8
0.91


### Question 2.2
Using `max_depth=3`, showing the accuracy score of test data by `criterion=gini` and `criterion=entropy`, respectively.


In [9]:
# clf_gini = DecisionTree(criterion='gini', max_depth=3)
# clf_entropy = DecisionTree(criterion='entropy', max_depth=3)

In [114]:
clf_gini = build_tree(training_data, 'gini', 3)
clf_entropy = build_tree(training_data, 'entropy', 3)

In [140]:
print(test_acc(testing_data, clf_gini))
print(test_acc(testing_data, clf_entropy))

0.8
0.8


- Note: Your decisition tree scores should over **0.7**. It may suffer from overfitting, if so, you can tune the hyperparameter such as `max_depth`
- Note: You should get the same results when re-building the model with the same arguments,  no need to prune the trees
- Hint: You can use the recursive method to build the nodes


## Question 3
Plot the [feature importance](https://sefiks.com/2020/04/06/feature-importance-in-decision-trees/) of your Decision Tree model. You can get the feature importance by counting the feature used for splitting data.

- You can simply plot the **counts of feature used** for building tree without normalize the importance. Take the figure below as example, outlook feature has been used for splitting for almost 50 times. Therefore, it has the largest importance

![image](https://i2.wp.com/sefiks.com/wp-content/uploads/2020/04/c45-fi-results.jpg?w=481&ssl=1)

## Question 4
implement the AdaBooest algorithm by using the CART you just implemented from question 2 as base learner. You should implement one arguments for the AdaBooest.
1. **n_estimators**: The maximum number of estimators at which boosting is terminated

In [343]:
class AdaBoost():
    def __init__(self, n_estimators):
        return None

### Question 4.1
Show the accuracy score of test data by `n_estimators=10` and `n_estimators=100`, respectively.


## Question 5
implement the Random Forest algorithm by using the CART you just implemented from question 2. You should implement three arguments for the Random Forest.

1. **n_estimators**: The number of trees in the forest. 
2. **max_features**: The number of random select features to consider when looking for the best split
3. **bootstrap**: Whether bootstrap samples are used when building tree


In [11]:
class RandomForest():
    def __init__(self, n_estimators, max_features, boostrap=True, criterion='gini', max_depth=None):
        return None

### Question 5.1
Using `criterion=gini`, `max_depth=None`, `max_features=sqrt(n_features)`, showing the accuracy score of test data by `n_estimators=10` and `n_estimators=100`, respectively.


In [12]:
clf_10tree = RandomForest(n_estimators=10, max_features=np.sqrt(x_train.shape[1]))
clf_100tree = RandomForest(n_estimators=100, max_features=np.sqrt(x_train.shape[1]))

### Question 5.2
Using `criterion=gini`, `max_depth=None`, `n_estimators=10`, showing the accuracy score of test data by `max_features=sqrt(n_features)` and `max_features=n_features`, respectively.


In [13]:
clf_random_features = RandomForest(n_estimators=10, max_features=np.sqrt(x_train.shape[1]))
clf_all_features = RandomForest(n_estimators=10, max_features=x_train.shape[1])

- Note: Use majority votes to get the final prediction, you may get slightly different results when re-building the random forest model

### Question 6.
Try you best to get highest test accuracy score by 
- Feature engineering
- Hyperparameter tuning
- Implement any other ensemble methods, such as gradient boosting. Please note that you cannot call any package. Also, only ensemble method can be used. Neural network method is not allowed to used.

In [ ]:
from sklearn.metrics import accuracy_score

In [4]:
y_test = test_df['target']

In [ ]:
y_pred = your_model.predict(x_test)

In [ ]:
print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))

## Supplementary
If you have trouble to implement this homework, TA strongly recommend watching [this video](https://www.youtube.com/watch?v=LDRbO9a6XPU), which explains Decision Tree model clearly. But don't copy code from any resources, try to finish this homework by yourself! 